### Tutorial 2: Trajectory Optimization for Ergodic Control

References:

[1] *Miller, L.M., Silverman, Y., MacIver, M.A. and Murphey, T.D.*, 2015. **Ergodic exploration of distributed information**. IEEE Transactions on Robotics, 32(1), pp.36-52. [[Link](https://ieeexplore.ieee.org/abstract/document/7350162)]

[2] *Miller, L.M. and Murphey, T.D.*, 2013, June. **Trajectory optimization for continuous ergodic exploration**. In 2013 American Control Conference (pp. 4196-4201). IEEE. [[Link](https://ieeexplore.ieee.org/abstract/document/6580484)]

In [1]:
import numpy as np 
np.set_printoptions(precision=4)
from scipy.stats import multivariate_normal as mvn
from tqdm import tqdm

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 3
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['axes.titlepad'] = 8.0
mpl.rcParams['xtick.major.size'] = 6
mpl.rcParams['xtick.major.width'] = 3
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.major.size'] = 6
mpl.rcParams['ytick.major.width'] = 3
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['lines.markersize'] = 5
mpl.rcParams['lines.linewidth'] = 5
mpl.rcParams['legend.fontsize'] = 15

#### Quick recap on trajectory optimization with iLQR

Consider the following standard formula for an optimal control problem:

$$
\begin{align}
u^*(t) & = {\arg\min}_{u(t)} \int_{0}^{T} l(x(t), u(t)) dt + m(x(T)) \\
s.t. & \quad \dot{x}(t) = f(x(t), u(t)), \quad x(0) = x_0
\end{align}
$$

The iterative linear quadratic regular (iLQR) algorithm follows a gradient descent approach to find a locally optimal solution. Given the current estimation of the control $u(t)$, at each iteration, iLQR finds the descent direction $v(t)$ by solving the following ODEs:

$$
\begin{align}
    & B(t)^\top p(t) + b(t) + R_v^\top v(t) = 0 \\
    & \dot{p}(t) = -A(t)^\top p(t) - a(t) - Q_z^\top z(t) \\
    & \dot{z}(t) = A(t) z(t) + B(t) v(t),
\end{align}
$$ 

with the intial and terminal conditions:

$$
\begin{align}
    z(0) = 0, \quad p(T) = \frac{d}{dx} m(x(T))
\end{align}
$$

where

$$
\begin{align}
    A(t) & = \frac{d}{dx} f(x(t), u(t)) \\
    B(t) & = \frac{d}{du} f(x(t), u(t)) \\
    a(t) & = \frac{d}{dx} l(x(t), u(t)) \\
    b(t) & = \frac{d}{du} l(x(t), u(t)),
\end{align}
$$

and $Q_z$ and $R_v$ are user-defined regularization parameters.

One can solve the ODEs as a two-point boundary value problem or by solving the Riccati equation. We provide a template for the iLQR algorithm below.

In [2]:
class iLQR_template:
    def __init__(self, dt, tsteps, x_dim, u_dim, Q_z, R_v) -> None:
        self.dt = dt 
        self.tsteps = tsteps 

        self.x_dim = x_dim 
        self.u_dim = u_dim

        self.Q_z = Q_z 
        self.Q_z_inv = np.linalg.inv(Q_z)
        self.R_v = R_v 
        self.R_v_inv = np.linalg.inv(R_v)

        self.curr_x_traj = None 
        self.curr_y_traj = None

    def dyn(self, xt, ut):
        raise NotImplementedError("Not implemented.")

    def step(self, xt, ut): 
        """RK4 integration"""
        k1 = self.dt * self.dyn(xt, ut)
        k2 = self.dt * self.dyn(xt + k1/2.0, ut)
        k3 = self.dt * self.dyn(xt + k2/2.0, ut)
        k4 = self.dt * self.dyn(xt + k3, ut)

        xt_new = xt + (k1 + 2.0*k2 + 2.0*k3 + k4) / 6.0 
        return xt_new 
    
    def traj_sim(self, x0, u_traj):
        x_traj = np.zeros((self.tsteps, self.x_dim))
        xt = x0.copy()
        for t_idx in range(self.tsteps):
            xt = self.step(xt, u_traj[t_idx])
            x_traj[t_idx] = xt.copy()
        return x_traj
    
    def loss(self):
        raise NotImplementedError("Not implemented.")
    
    def get_At_mat(self, t_idx):
        raise NotImplementedError("Not implemented.")
    
    def get_Bt_mat(self, t_idx):
        raise NotImplementedError("Not implemented.")

    def get_at_vec(self, t_idx):
        raise NotImplementedError("Not implemented.")
    
    def get_bt_vec(self, t_idx):
        raise NotImplementedError("Not implemented.")

    # the following functions are utilities for solving the Riccati equation
    def P_dyn_rev(self, Pt, At, Bt, at, bt):
        return Pt @ At + At.T @ Pt - Pt @ Bt @ self.R_v_inv @ Bt.T @ Pt + self.Q 
    
    def P_dyn_step(self, Pt, At, Bt, at, bt):
        k1 = self.dt * self.P_dyn_rev(Pt, At, Bt, at, bt)
        k2 = self.dt * self.P_dyn_rev(Pt+k1/2, At, Bt, at, bt)
        k3 = self.dt * self.P_dyn_rev(Pt+k2/2, At, Bt, at, bt)
        k4 = self.dt * self.P_dyn_rev(Pt+k3, At, Bt, at, bt)

        Pt_new = Pt + (k1 + 2.0*k2 + 2.0*k3 + k4) / 6.0 
        return Pt_new 
    
    def P_traj_revsim(self, PT, A_list, B_list, a_list, b_list):
        P_traj_rev = np.zeros((self.tsteps, self.dim, self.dim))
        P_curr = PT.copy()
        for t in range(self.tsteps):
            At = A_list[-1-t]
            Bt = B_list[-1-t]
            at = a_list[-1-t]
            bt = b_list[-1-t]

            P_new = self.P_dyn_step(P_curr, At, Bt, at, bt)
            P_traj_rev[t] = P_new.copy()
            P_curr = P_new 
        
        return P_traj_rev

    def r_dyn_rev(self, rt, Pt, At, Bt, at, bt):
        return (At - Bt @ self.R_v_inv @ Bt.T @ Pt).T @ rt + at - Pt @ Bt @ self.R_v_inv @ bt

    def r_dyn_step(self, rt, Pt, At, Bt, at, bt):
        k1 = self.dt * self.r_dyn_rev(rt, Pt, At, Bt, at, bt)
        k2 = self.dt * self.r_dyn_rev(rt+k1/2, Pt, At, Bt, at, bt)
        k3 = self.dt * self.r_dyn_rev(rt+k2/2, Pt, At, Bt, at, bt)
        k4 = self.dt * self.r_dyn_rev(rt+k3, Pt, At, Bt, at, bt)

        rt_new = rt + (k1 + 2.0*k2 + 2.0*k3 + k4) / 6.0 
        return rt_new
    
    def r_traj_revsim(self, rT, P_list, A_list, B_list, a_list, b_list):
        r_traj_rev = np.zeros((self.tsteps, self.dim))
        # r_curr = np.zeros(self.dim)
        r_curr = rT
        for t in range(self.tsteps):
            Pt = P_list[-1-t]
            At = A_list[-1-t]
            Bt = B_list[-1-t]
            at = a_list[-1-t]
            bt = b_list[-1-t]

            r_new = self.r_dyn_step(r_curr, Pt, At, Bt, at, bt)
            r_traj_rev[t] = r_new.copy()
            r_curr = r_new 

        return r_traj_rev

    def z_dyn(self, zt, Pt, rt, At, Bt, bt):
        return At @ zt + Bt @ self.z2v(zt, Pt, rt, Bt, bt)
    
    def z_dyn_step(self, zt, Pt, rt, At, Bt, bt):
        k1 = self.dt * self.z_dyn(zt, Pt, rt, At, Bt, bt)
        k2 = self.dt * self.z_dyn(zt+k1/2, Pt, rt, At, Bt, bt)
        k3 = self.dt * self.z_dyn(zt+k2/2, Pt, rt, At, Bt, bt)
        k4 = self.dt * self.z_dyn(zt+k3, Pt, rt, At, Bt, bt)

        zt_new = zt + (k1 + 2.0*k2 + 2.0*k3 + k4) / 6.0 
        return zt_new

    def z_traj_sim(self, z0, P_list, r_list, A_list, B_list, b_list):
        z_traj = np.zeros((self.tsteps, self.dim))
        z_curr = z0.copy()

        for t in range(self.tsteps):
            Pt = P_list[t]
            rt = r_list[t]
            At = A_list[t]
            Bt = B_list[t]
            bt = b_list[t]

            z_new = self.z_dyn_step(z_curr, Pt, rt, At, Bt, bt)
            z_traj[t] = z_new.copy()
            z_curr = z_new
        
        return z_traj
    
    def z2v(self, zt, Pt, rt, Bt, bt):
        return -self.R_v_inv @ Bt.T @ Pt @ zt - self.R_v_inv @ Bt.T @ rt - self.R_v_inv @ bt

    def get_descent(self, x0, u_traj):
        # forward simulate the trajectory
        x_traj = self.traj_sim(x0, u_traj)
        self.curr_x_traj = x_traj.copy()
        self.curr_u_traj = u_traj.copy()

        # sovle the Riccati equation backward in time
        A_mat_list = np.zeros((self.tsteps, self.x_dim, self.x_dim))
        B_mat_list = np.zeros((self.tsteps, self.x_dim, self.u_dim))
        a_vec_list = np.zeros((self.tsteps, self.x_dim))
        b_vec_list = np.zeros((self.tsteps, self.u_dim))

        for t_idx in range(self.tsteps):
            A_mat_list[t_idx] = self.get_At_mat(t_idx)
            B_mat_list[t_idx] = self.get_Bt_mat(t_idx)
            a_vec_list[t_idx] = self.get_at_vec(t_idx)
            b_vec_list[t_idx] = self.get_bt_vec(t_idx)
        
        PT = np.zeros((self.x_dim, self.x_dim))
